In [4]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm.notebook import tqdm

In [71]:
# Load our data.
df = pd.read_csv("data/Normalized_Purchase_Card_Transactions.csv")
df_agencies = pd.read_csv("data/List_Agencies.csv")
df_vendors = pd.read_csv("data/List_Vendors.csv")

In [72]:
df_vendors

VENDOR_NUM             VENDOR_NAME
0               0                     NaN
1               1                       A
2               2                    AAAA
3               3  AAACE_AMERICAN_ASSOCIA
4               4    AAAGA_WORLD_CONGRESS
...           ...                     ...
38095       38095      __MANASSAS__CAPTRI
38096       38096           __OIL_PRO_INC
38097       38097        __PENTAGON_CENTR
38098       38098       __UPPER_MARLBORO_
38099       38099             ___TH_ST_SW

[38100 rows x 2 columns]

In [6]:
blank = np.zeros((df['AGENCY_NUM'].max() + 1, df['VENDOR_NAME_NUM'].max() + 1))

for index, row in tqdm(df.iterrows(), total = df.shape[0]):
    blank[row['AGENCY_NUM'], row['VENDOR_NAME_NUM']] += row['TRANSACTION_AMOUNT_NORM']

np.count_nonzero(blank)

  0%|          | 0/390109 [00:00<?, ?it/s]

44494

In [7]:
wordCounts = pd.DataFrame(blank)

bags = {i: df[df['AGENCY_NUM'] == i]['VENDOR_NAME_NUM'].unique() for i in df['AGENCY_NUM'].unique()}

In [25]:
bags

{34: array([37719, 21724,  4184, ..., 37241,   127, 17728]),
 43: array([ 6406, 19485, 12057, ..., 34629, 15597, 20903]),
 24: array([36174,  1390,  9816, ..., 37372, 21143, 21357]),
 59: array([32645, 11555, 32622, 35086, 30083, 20960, 21008, 16606, 33804,
        35775, 23430, 27590,  9913, 22092, 34803,  8773, 35693, 30128,
        33906, 36576,   775, 34339, 27920, 23402, 19924, 28876, 23483,
        23411,  9230,  5571, 33034, 19926, 34687, 13037,  5848,  7028,
        27054, 23614, 36572, 16929,   686, 30361, 27593,  9483, 22128,
        21026, 34688, 20966, 22690,   707, 32648, 36513,  4266]),
 67: array([12917,  1298,  4129, 37782,   370, 10885,  9652, 33805,  5112,
        37940, 35730, 35795, 35478,  7069,  4242,  1027, 23177,  9643,
        23749, 35408, 30675, 16937, 30083, 13064, 33327, 16684, 33112,
        10321,  7031,  1390, 19066,  9775, 13069, 37436, 37455,   154,
        34573, 29779,  6710, 16780, 37115, 33418, 20930, 27986,  3874,
         8773, 35693, 23259, 1028

In [8]:
def tf(counts):
    tfDF = pd.DataFrame()
    
    for index, row in counts.iterrows():
        tfDF[index] = row.div(row.sum())
        
    return tfDF

tf = pd.DataFrame(tf(wordCounts)).T

In [9]:
idf = tf.apply(lambda x: np.log((df['AGENCY_NUM'].max() + 1) / (np.count_nonzero(x.to_numpy())+1)), axis=0)

In [10]:
prod = tf*idf

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(prod).astype("float")
for i in range(len(similarity)):
    similarity[i][i] = 0

In [12]:
# Find most similar vendors
mostsimilar = np.argmax(similarity, axis=1)
mostsimilar_verbose = np.array([df_agencies.iloc[i][1] for i in mostsimilar])
mostsimilar_vendors = np.array([bags[i] for i in mostsimilar], dtype = object)

In [13]:
grouped = df.groupby(['AGENCY_NUM', 'VENDOR_NAME_NUM'], as_index = False)['TRANSACTION_AMOUNT'].agg('sum')
topn = grouped.sort_values(['AGENCY_NUM','TRANSACTION_AMOUNT'], ascending=False).groupby('AGENCY_NUM').head(5)

In [14]:
descriptions = df.loc[:, ['VENDOR_NAME_NUM', 'TRANSACTION_AMOUNT', 'MCC_DESCRIPTION']]
descriptions['MCC_DESCRIPTION'] = descriptions['MCC_DESCRIPTION'].str.replace('Not Elsewhere Classified', '').str.replace('NOT ELSEWHERE CLASSIFIED', '')
descriptions['MCC_DESCRIPTION_LIST'] = descriptions['MCC_DESCRIPTION'].str.split(r',\s*(?![^()]*\))')

In [15]:
exploded = descriptions.explode('MCC_DESCRIPTION_LIST').replace(' ', np.nan).dropna(0)

In [16]:
grouped_descriptions = exploded.groupby('MCC_DESCRIPTION_LIST').sum('TRANSACTION_AMOUNT')
topn_descriptions = pd.DataFrame(grouped_descriptions['TRANSACTION_AMOUNT'].groupby('MCC_DESCRIPTION_LIST', group_keys = False).nlargest(25))

In [17]:
# TOP N
topn

AGENCY_NUM  VENDOR_NAME_NUM  TRANSACTION_AMOUNT
48074          88            24206           554789.05
48171          88            29793           221999.31
47991          88            20930           129553.78
48203          88            30791           116204.61
48252          88            32622           103940.03
...           ...              ...                 ...
28              0            32622            16490.67
13              0            10283             3341.96
22              0            14975             2050.00
32              0            34339             1982.25
15              0            10897             1934.62

[445 rows x 3 columns]

In [18]:
# topn_descriptions
descriptions

VENDOR_NAME_NUM  TRANSACTION_AMOUNT  \
0                 37719              229.50   
1                  6406             3147.33   
2                 36174               54.00   
3                 32645              137.35   
4                 12917             1340.80   
...                 ...                 ...   
390104             4266               59.96   
390105            17784              480.00   
390106            17784             2112.00   
390107            14337             3689.01   
390108            20930             3360.33   

                                          MCC_DESCRIPTION  \
0                                   Industrial Supplies,    
1               Stationery, Office & School Supply Stores   
2       Motion Pictures/Video Tape Production&Distribu...   
3       Stationery,Office Supplies,Printing/Writing Paper   
4               Stationery, Office & School Supply Stores   
...                                                   ...   
390104                                        BOOK STORES   
390105                     ELECTRICAL PARTS AND EQUIPMENT   
390106                     ELECTRICAL PARTS AND EQUIPMENT   
390107                             COMMERCIAL EQUIPMENT     
390108  COMPUTERS  COMPUTER PERIPHERAL EQUIPMENT  SOFT...   

                                     MCC_DESCRIPTION_LIST  
0                                 [Industrial Supplies, ]  
1             [Stationery, Office & School Supply Stores]  
2       [Motion Pictures/Video Tape Production&Distrib...  
3       [Stationery, Office Supplies, Printing/Writing...  
4             [Stationery, Office & School Supply Stores]  
...                                                   ...  
390104                                      [BOOK STORES]  
390105                   [ELECTRICAL PARTS AND EQUIPMENT]  
390106                   [ELECTRICAL PARTS AND EQUIPMENT]  
390107                           [COMMERCIAL EQUIPMENT  ]  
390108  [COMPUTERS  COMPUTER PERIPHERAL EQUIPMENT  SOF...  

[390109 rows x 4 columns]

In [49]:
descriptions_per_agency = {}
for i in range(df_agencies.shape[0]):
    top_5_vendors = topn[topn['AGENCY_NUM'] == i]['VENDOR_NAME_NUM'].to_numpy()
    
    desc = []
    for j in top_5_vendors:
        desc += np.unique(descriptions[descriptions['VENDOR_NAME_NUM'] == j]['MCC_DESCRIPTION_LIST'].to_numpy()).tolist()[0]
        
    descriptions_per_agency[i] = desc

print(descriptions_per_agency)

{0: ['COMMERCIAL EQUIPMENT  ', 'COMPUTERS  COMPUTER PERIPHERAL EQUIPMENT  SOFTWARE', 'Non-Durable Goods', '', 'Professional Services ', 'Business Services '], 1: ['COMPUTERS  COMPUTER PERIPHERAL EQUIPMENT  SOFTWARE', 'COMMERCIAL EQUIPMENT  ', 'Business Services ', 'Commercial Photography', 'Art & Graphics', 'Office and Commercial Furniture'], 2: ['Computer Maint&Repair Service', 'Not Elsewhere Class.', 'BOOK STORES', 'COMPUTERS  COMPUTER PERIPHERAL EQUIPMENT  SOFTWARE', 'MISCELLANEOUS PUBLISHING AND PRINTING', 'OFFICE  SCHOOL SUPPLY  AND STATIONERY STORES'], 3: ['PROFESSIONAL SERVICES-', 'COMMERCIAL EQUIPMENT  ', 'COMPUTERS  COMPUTER PERIPHERAL EQUIPMENT  SOFTWARE', 'Accounting', 'Auditing & Bookkeeping Services', 'COMPUTERS  COMPUTER PERIPHERAL EQUIPMENT  SOFTWARE'], 4: ['COMPUTERS  COMPUTER PERIPHERAL EQUIPMENT  SOFTWARE', 'STATIONERY-OFFICE SUPPLIES-PRINTING + WRITING PAP.', 'OFFICE PHOTOGRAPHIC PHOTOCOPY + MICROFILM EQUIPMT.', 'COMMERCIAL EQUIPMENT  ', 'DISCOUNT STORES'], 5: ['COMP

In [157]:
vendor_names = exploded.loc[:, ['VENDOR_NAME_NUM', 'MCC_DESCRIPTION_LIST']].dropna(0)
vendor_names['MCC_DESCRIPTION_LIST'] = vendor_names['MCC_DESCRIPTION_LIST'].str.upper()
df_vendors.join

def summarize(agency = 0):
    """
    Gives brief information on the agency and its most similar agency.
    """
    
    # gets the list of vendors this agency uses
    agency_bag = bags[agency]
    agency_bag_verbose = np.array([df_vendors.iloc[i][1] for i in agency_bag])
        
    # get the vendors of the most similar agency
    similar_agency_bag = bags[mostsimilar[agency]]
    similar_agency_bag_verbose = pd.DataFrame([df_vendors.iloc[i][1] for i in similar_agency_bag]).join(vendor_names)
    
    similar_agency_bag_verbose = similar_agency_bag_verbose
        
    topVendors = [x.strip().upper() for x in descriptions_per_agency[mostsimilar[agency]]]
    
    newVendors = []
    
    for index, row in similar_agency_bag_verbose.iterrows():
        if row['MCC_DESCRIPTION_LIST'] in topVendors:
            if row[0] not in agency_bag_verbose:
                newVendors.append(row[0])
        
#     print("Selected Agency:", df_agencies.iloc[agency][1])
#     print("Most Similar Agency:", mostsimilar_verbose[agency])
#     print()
#     print("Agency Vendors:", agency_bag_verbose)
#     print()
#     print("Most Similar Agency Vendors:", similar_agency_bag_verbose[0].to_numpy())
#     print()
#     print("Potential New Vendors:", newVendors)
    
    summaryDict = {
        'agency': df_agencies.iloc[agency][1],
        'agency_num': int(df_agencies.iloc[agency][0]),
        #'similar_agency': mostsimilar_verbose[agency].tolist(),
        #'agency_vendors': agency_bag_verbose.tolist(),
        #'similar_agency_vendors': similar_agency_bag_verbose[0].to_numpy().tolist(),
        'new_vendors': newVendors
    }
        
    return summaryDict

In [156]:
bigDict = {}

for i in tqdm(range(0, 89), total = 89):
    bigDict[i] = summarize(i)

  0%|          | 0/89 [00:00<?, ?it/s]

{'agency': 'Advisory Neighborhood Commission', 'agency_num': 0, 'similar_agency': 'Unknown New Agency', 'agency_vendors': ['STANDARD_OFFICE_SUPPLY', 'STAPLES_', 'FRYSCOM', 'THOMSON_WESTTCD', 'EIGDOTHOSTING', 'EIGIPOWER', 'BUDGET_TRUCK_RENTAL', 'RADIOSHACK_COR', 'GODADDYCOM', 'PROVANTAGE_CORP', 'MERITLINECOM', 'CABLEORGANIZER__', 'ADOBE_SYSTEMS_INC', 'PENN_CAMERA_', 'WALMART_', 'ADVERTISING', 'BARNES__NOBLE_', 'BLUANCDORG', 'HACKETT_AND_FAMILY_CAT', 'CORNER_BAKERY_', 'BEST_BUY_', 'STAPLES_DIRECT', 'AMERICAN_LEGAL_PUBLISH', 'CVSPHARMACY_', 'DRIPLANTRONICS', 'DUTCH_MILL_CATERING', 'AMAZON_MKTPLACE_PMTS', 'THE_WRIGHT_HOUSE_CO', 'DMI_DELL_HLTHCRREL', 'EIG', 'AMAZON_PRIME', 'AMZN_MKTP_US', 'FS_TECHSMITH', 'MICROSOFTSTORE'], 'similar_agency_vendors': ['STANDARD_OFFICE_SUPPLY', 'STANDARD_OFFICE_SUPPLY', 'SQ_RUSH_DELIVERY_MESS', 'SQ_RUSH_DELIVERY_MESS', 'METROPOLITAN_OFFICE_PR', 'NPDB_NPDBHIPDBHRSAG', 'NPDB_NPDBHIPDBHRSAG', 'NPDB_NPDBHIPDBHRSAG', 'HIPDB_NPDBHIPDBHRSA', 'HIPDB_NPDBHIPDBHRSA', 'O

{'agency': 'Board of Elections & Ethics', 'agency_num': 2, 'similar_agency': 'Office of Disability Rights', 'agency_vendors': ['STAPLES_', 'AJ_STATIONERS_OF_DC', 'SPECIALTYBOOKSCOM_', 'GATEKEEPER_INTERNET_MK', 'VIRGINIA_CLE', 'CAPITAL_SERVICES_AND_S', 'MATTHEW_BENDER__CO_IN', 'NATIONAL_MAILING_SYSTE', 'OFFICE_DEPOT_', 'OFFICE_OF_DOC_AND_ADMI', 'EASTERN_LIFT_TRUCK_CO', 'NATIONAL_POSTAL_FORUM', 'SOUTHWEST_DISTRIBUTION', 'PRESTO_THE_NOTARY_STO', 'TX_NICUSA_NATIONAL_EFI', 'ARNOLD_ENGRAVERS__COR', 'GRADUATE_SCHOOL_REG', 'ALMAMER_LAWER_MEDIA', 'ACCURATE_CONCEPTIONS_L', 'DRINUANCE', 'BARNESNOBLECOM', 'HASLER_RC_', 'USPS_QPS', 'EFILING_FOR_COURTS', 'AMERITEL_COMMUNICATION', 'REIMATTHEW_BENDER_CO', 'THE_DAILY_WASHINGTON_L', 'METROPOLITAN_OFFICE_PR', 'FEDEX_KINKOS_', 'INTAB_INC', 'AMAZON_MKTPLACE_PMTS', 'SYDAR_PRINTING_CORP', 'STANDARD_OFFICE_SUPPLY', 'THE_HOME_DEPOT_', 'SYDAR_PRINTING_CORPQPS', 'FASTSIGNS', 'FELLERS_INC', 'TWILIO', 'PITNEY_BOWES', 'SYDAR_OF_DC_LLC_QPS', 'THE_KOREA_TIMES_DC', 'A

{'agency': 'Child & Family Services Administration', 'agency_num': 4, 'similar_agency': 'DEPUTY MAYOR FOR PUBLIC SAFETY & JUSTICE', 'agency_vendors': ['CVS_PHARMACY__Q', 'POSTER_COMPLIANCE_CENT', 'DRINUANCE', 'METROPOLITAN_OFFICE_PR', 'STANDARD_OFFICE_SUPPLY', 'TRAVEL_INSURANCE_POLIC', 'USAIRWAY_', 'THE_HOME_DEPOT_', 'TRICOM_INC_OF_WISCONS', 'GREYHOUND_LINES_', 'TARGET_', 'CANON_FINANCIAL_SERVIC', 'USPS_QPS', 'CHILD_WELFARE_CVENT', 'ABES_TRANSPORTATION', 'FAITH_MGMT_CONSULTING', 'CAPT_ORG', 'QSR_INTERNATIONAL', 'GSA_OFFICE_SUPPLIES_IN', 'GLP_INC', 'TONER_EXPRESS_USA_INC', 'NIGP', 'FRED_PRYOR_SEMINARS', 'LASER_ART_INC', 'STAR_OFFICE_PRODUCTS_I', 'GLIGREYHOUNDLINES', 'AMTRAK_TEL', 'RED_ROOF_INNS', 'RITE_AID_STORE_Q', 'SLOAN_WILLIAMS_PROMOTI', 'NEWSEUM', 'DROPIO_INC', 'OLD_NAVY_', 'VCNMARYLAND_VITAL_REC', 'VCNVIRGINIA_VITAL_REC', 'DOCUMENT_MANAGERS', 'HI_TECH_SOLUTIONS_INC', 'DC_GOVT_CONSUMERREG', 'IMPACT_OFFICE_PRODUCTS', 'ULINE_SHIP_SUPPLIES', 'NIGPORG', 'COMCAST_OF_WASHINGTON', 'GELBER

{'agency': 'Commission on Arts & Humanities', 'agency_num': 7, 'similar_agency': 'Office on Aging', 'agency_vendors': ['METROPOLITAN_OFFICE_PR', 'AMERICANS_FOR_THE_ARTS', 'CAPITAL_SERVICES_AND_S', 'CREATION_ENGINE_INC', 'MAC_UPGRADES', 'CONSTANT_CONTACT', 'R_STREET_SELF_STORAGE', 'RANSAR_BUSINESS_SERVIC', 'TARGET_', 'ORANGE__INC', 'STAPLES', 'STAPLES_DIRECT', 'QUICK_MESSENGER_SERVIC', 'QUALITY_LOGO_PRODUCTS', 'USPS__QQQ', 'JARBOE_PRINTING_CO_INC', 'RED_COATS', 'KINKOSCOM_PK', 'COMCAST_CABLEVISION_DI', 'STAPLES_', 'DATAWATCH_SYSTEMS_INC', 'UTRECHT_ART_SUPPLIES_', 'AFFORDABLE_SIGNSBANNE', 'VISTAPRVISTAPRINTCOM', 'MULLENS_TRANSFER__ST', 'WASHINGTON_BIZ_JOURNAL', 'AMAZONCOM', 'CAPITAL_EXHIBITS', 'REGCENTER_FOR_NONPROF', 'GOFEDEX_', 'FEDEX_KINKOS_', 'BEST_BUY_', 'PAYPAL_PEACE_TILES', 'MARSHALLS_', 'TGTTARGETCOM', 'LUERZERS_ARCHIVE_INC', 'NETWORK_FOR_GOOD', 'CDW_GOVERNMENT', 'THE_BODY_SHOP_', 'RITE_AID_STORE_Q', 'FLASH_DEALER_LLC', 'TLFLEE_FLOWERS__CAR', 'WDCW_BROADCASTING_INC', 'FEDEX_', 'W

...

{'agency': 'Office on Asian/Pacific Affairs', 'agency_num': 83, 'similar_agency': 'Department on Disability Services', 'agency_vendors': ['LASER_ART_INC', 'SENODA_INC', 'SOUTH_ASIAN_AMERICAN', 'SHARP_BUSINESS_SYS_DC', 'WSHPOSTCLEACC_AR_CLEA', 'DEMOCRACYINACTIONORG', 'REGCENTER_FOR_NONPROF', 'PAYPAL_DAKSHINADAN', 'BRENTWORKS_INC', 'JARBOE_PRINTING_CO_INC', 'POTBELLY_', 'TARGET_', 'BARNES__NOBLE_Q', 'TONY_CHENG_S_SEAFOOD_R', 'METRO_CNTR_TKT_OFFICE', 'GIMGA_GROUP', 'AC_GRAPHICS', 'VISUAL_AIDS_ELECTRONIC', 'STANDARD_OFFICE_SUPPLY', 'GIANT_FOOD_INC_', 'MULTICULTURAL_COMMUNIT', 'TRANSEMANTICS_INC', 'MVS_INCCREDIT_MVS', 'WMATA_METRO_CENTER_QPS', 'GELBERG_SIGNS', 'STAPLES_', 'FEDEX_KINKOS_QPS', 'PAYPAL', 'ALLIED_PARTY_RENTALS', 'DUPONT_COMPUTER_INC', 'GRANTWRITIN', 'SAFEWAY_STORE_QPS', 'MOUNT_ROYAL_PRINTING_CO_INC', 'WINZIP_COMPUTING_SL', 'TONER_EXPRESS_USA_INC', 'SAFEWAY_STOREQPS', 'EDWEEKORG', 'WASHINGTON_PARKS__PEO', 'WMATA_METRO_CENTER_BUL', 'CTCCONSTANTCONTACTCO', 'THE_ARC', 'PAYPAL_CHINE

{'agency': 'Public Service Commission', 'agency_num': 85, 'similar_agency': "OFFICE OF PEOPLE'S COUNSEL", 'agency_vendors': ['ELSEVIER_LTD', 'MMI_INC', 'PROGRESSIVE_BUSIN', 'NATL_STDNT_CLEARINGHOU', 'THOMSON_WESTTCD', 'TALX_CORPORATION', 'AMTRAK_CO', 'WW_GRAINGER', 'GRANTWRITIN', 'LIFE_INFINITE_LLC', 'NATL_ASSN_REG_UTIL_COM', 'KASTLE_SYSTEMS', 'WHITAKERBROTHERSCOM', 'CAPITAL_SERVICES_AND_S', 'USAIRWAY_', 'ABC_TECHNICAL_SOLUTION', 'MICHIGAN_STATE_UNIVERS', 'DMI_DELL_KGOVT', 'WASHINGTON_ENGRAVING_C', 'LT_PRINTING_COMPANY', 'GESTALT_PARTNERS_LLC', 'GHI_LLC', 'WL_GARY_CO', 'SOUTHWES_', 'SUPERSHUTTLE_EXECUCARB', 'PAYPAL_GREATERWASH', 'AIRTRANAI_', 'XEROX_CORP_XCSRBO', 'AMERICAN_', 'SOLAR_ENERGY_TRADE_SHO', 'ENERGY_BAR_ASSOCIA', 'EUCI', 'NATIONAL_MAILING_SYSTE', 'PACIFIC_MARKETING_LLC', 'TPW_CONSULTANTS', 'NBA_OFFICE_PRODUCTS_I', 'DELTA_', 'AIRLINES_', 'GRADUATE_SCHOOL_REG', 'IWP_NEWSLETTERS', 'COAAIRLINETAXESFEES', 'DRCC_EVENT_ADS_MEM', 'ALASKA_A_', 'UNITED_', 'ROCKHURST_UNIVERSITY_C', 'FRO

{'agency': 'Serve DC', 'agency_num': 86, 'similar_agency': 'Office of the Mayor', 'agency_vendors': ['STAPLES_', 'METROPOLITAN_OFFICE_PR', 'AMERICAN_', 'STANDARD_OFFICE_SUPPLY', 'NATIONAL_YOUTH_LEADERS', 'PAYPAL_SETTANNI_CO', 'THE_HAMILTON_GROUP', 'NTL_HURRICANE_CONFEREN', 'YNPN_DC', 'WMATA_METRO_CENTER_BUL', 'WASHINGTON_PHOTO_COPY', 'SENODA_INC', 'NATL_GRANTS_MANAGEMENT', 'POINTS_OF_LIGHT_NET', 'THE_HOME_DEPOT_', 'LIFE_INFINITE_LLC', 'UNITED_', 'PEMBROKE_WEST', 'NATIONAL_VOAD', 'FRONTIER_', 'CAPITAL_SERVICES_AND_S', 'REGCENTER_FOR_NONPROF', 'DELTA_', 'USAIRWAY_', 'CAPITAL_COMMUNITY_NEWS', 'FRAGERS_HARDWARE', 'PROPAC', 'HOTEL_MONACO_SALT_LAKE', 'HILTON_HOTELS', 'CPR_AED__SAFETY_EDUCA', 'NWA_AIR_', 'COLORNET_PRINTING_AND', 'HANDSINMOTI', 'MINUTEMAN_PRESS', 'PAYPAL_PATHWAYSLLC', 'JIMMIE_MUSCATELLOS', 'POTBELLY_', 'TARGET_', 'WASH_METRORAIL', 'AP_BOOKSTORECOM', '_EXTRA_SPACE_STORA', 'ACE_PRESS', 'PAYPAL_BROWNM', 'SOUTHWES_', 'AMAZONCOM', 'CVS_PHARMACY__Q', 'INTERNATIONAL_ASSOCIAT', 'PAYPA

{'agency': 'University of the District of Columbia', 'agency_num': 87, 'similar_agency': 'District of Columbia Public Schools', 'agency_vendors': ['UDC_BOOKSTORE_', 'COLONIAL_PARKINGMAIN', 'LASER_ART_INC', 'BRAY_AND_SCARFF_', 'OFFICE_DEPOT_', 'MAGNA_PUBLICATIONS_IN', 'ATLANTIC_ELECTRIC_SUPP', 'CCAS', 'WMATA_METRO_CENTER_BUL', 'SIGMA_ALDRICH_US', 'APPLE_STORE_R', 'AMAZONCOM', 'AMZAMAZON_PAYMENTS', 'FISHER_SCI_CHI', 'TED_PELLA_INC', 'WW_GRAINGER', 'NASULGC', 'NATIONAL_ASSOCIATION_O', 'SETON_NAME_PLATE_COMPA', 'COLORNET_PRINTING_AND', 'CAREERBUILDER_COM', 'THE_UPS_STORE_', 'CENTRAL_WHOLESALER_SVS', 'CRUCIALCOMLEXARMEDIA', 'DURON_', 'THE_HOME_DEPOT_', 'MIDDLE_STATES_ASSOC_OF', 'MSCJL_INDUSTRIAL_SUP', 'CWOLCOM', 'PARTY_PLUS', 'CRAIGSLIST_ORG', 'ATS_TOTAL_SPORTS', 'WSGWILSON_SPTG_GDS', 'STAPLES_DIRECT', 'OFFENBACHERS__ROCKVI', 'FERGUSON_ENT_', 'BABOLAT_VS_NORTH_AMERI', 'WWWNEWEGGCOM', 'AACTEINTERNET', 'PAYPAL_DER_DONATIO', 'BED_BATH__BEYOND_', 'WORKSHOP_INC', 'FRAGERS_JUST_ASK_RENT', 'CUSTOM

{'agency': 'Unknown New Agency', 'agency_num': 88, 'similar_agency': 'Office of the Mayor', 'agency_vendors': ['STANDARD_OFFICE_SUPPLY', 'SQ_RUSH_DELIVERY_MESS', 'METROPOLITAN_OFFICE_PR', 'NPDB_NPDBHIPDBHRSAG', 'HIPDB_NPDBHIPDBHRSA', 'OFFICE_DEPOT_', 'STAPLS', 'BESTBUYMKTPLACE', 'LASER_ART_INC', 'THE_P_CONFERENCE', 'APEX_SOFTWARE', 'DTVDIRECTV_SERVICE', 'ZAGG', 'PACERIR', 'BESTBUYCOM_', 'CAPITAL_SERVICES__SUP', 'USPS_', 'PROJECT_MGMT_INSTITUTE', 'PEMBROKE_WEST', 'WHATTOEXPECT_ORG', 'SENODA_INC', 'STAPLES_', 'FEDEX_', 'STAR_OFFICE_PRODUCTS', 'LEXISNEXIS_RISK_MAN', 'CAPITAL_REPORTING_COMP', 'GRADUATE_SCHOOL_REG', 'RESIDEX_LLC', 'DMI_DELL_KGOVT', 'FAITH_MGMT_CONSULTING', 'AMAZONCOM', 'JTF_BUSINESS_SYSTEMS', 'TC_GNEIL', 'GWAFP', 'RADIOSHACK_', 'HOBBYLOBBY_', 'CDW_GOVERNMENT', 'MINDJET', 'TELESTARZ_LLC', 'THE_PEW_CHARITABLE', 'THE_HOME_DEPOT_', 'WB_MASON_CO_INC', 'CLINTON_DOLLAR_STORE', 'RINGCENTRAL_INC', 'WHITAKERBROTHERSCOM', 'APLAPPLEONLINESTOREUS', 'GOVERNMENT_FINANCE_OFF', 'BRANDERSCOM

In [158]:
import json

with open("results.json", "w") as outfile:  
    json.dump(bigDict, outfile) 